In [ ]:
#!/usr/bin/env python3
"""open_cv.ipynb
James Gardner 2019

various basic python3-openCV things
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [ ]:
def zoom(frame,r=0.5):
    """resizes frame while maintaining aspect ratio"""
    rheight = int(frame.shape[0]*r)
    rwidth  = int(frame.shape[1]*r)
    # caution: order of width, height in resize call
    return cv2.resize(frame,(rwidth,rheight))

In [ ]:
def fourier_plot(signal, fps, out_plot_name, out_plot_title=''):
    """plots fourier spectrum of time series"""
    # implementation from tracker_time_series
    # after_first loses a frame
    signal_frames = len(signal)
    total_time = (signal_frames-2)/fps
    t = np.linspace(0,total_time,signal_frames)
    dt = t[1] - t[0]

    yf = np.fft.fft(signal)
    # normalised-absolute value of FT'd signal
    nrm_abs_yf = 2/signal_frames*np.abs(yf)
    freq_scale = np.fft.fftfreq(len(yf),dt)
    # np.fft.fftfreq outputs 0 to +inf then -inf to 0, so :N//2 gets +ve side; wild!
    freq_prob = nrm_abs_yf[:signal_frames//2]

    fig, (ax0,ax1) = plt.subplots(2,figsize=(14,14))
    ax0.plot(t,signal)
    ax0.set(title='signal: {}'.format(out_plot_title),ylabel='signal strength',xlabel='time, t')
    # filter out the average value gives magnitude of zero freq term
    ax1.plot(freq_scale[:signal_frames//2][2:],freq_prob[2:])
    ax1.set(title='discrete FFT',ylabel='freq strength in signal',xlabel='frequency, f')
    # REMEMBER TO CHANGE OUT FILENAME
    plt.savefig(out_plot_name,bbox_inches='tight')
    plt.clf()

In [ ]:
def series_at_point(filename, point=None, return_series=False,
                    produce_plot=False, out_plot_name='tmp.pdf'):
    """green channel intensity at point in the interference pattern
    also plots time series and spectrum thereof
    aka: run_dog_run, won't produce/return anything unless you tell it to"""
    cap = cv2.VideoCapture(filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total=total_frames-1)

    point_intensity = []
    ret, frame = cap.read()
    # default to centre of frame
    if point is None:
        point = tuple([int(i/2) for i in frame.shape[:2]])

    while ret:
        # centre time series
        # use green channel since it has the greatest weight anyway
        # https://en.wikipedia.org/wiki/Relative_luminance
        point_intensity.append(frame[point][1])

        ret, frame = cap.read()
        pbar.update(1)

    pbar.close()
    cap.release()

    if produce_plot:
        fourier_plot(point_intensity, fps, out_plot_name, filename)
    if return_series:
        return point_intensity, fps

In [ ]:
def fourier_spectrum(signal, fps):
    """returns fourier spectrum of time series"""
    # implementation from tracker_time_series
    # after_first loses a frame
    signal_frames = len(signal)
    total_time = (signal_frames-2)/fps
    t = np.linspace(0,total_time,signal_frames)
    dt = t[1] - t[0]

    yf = np.fft.fft(signal)
    # normalised-absolute value of FT'd signal
    nrm_abs_yf = 2/signal_frames*np.abs(yf)
    freq_scale = np.fft.fftfreq(len(yf),dt)
    # np.fft.fftfreq outputs 0 to +inf then -inf to 0, so :N//2 gets +ve side; wild!
    freq_prob = nrm_abs_yf[:signal_frames//2]
    
    return freq_prob, freq_scale[:signal_frames//2]

In [ ]:
# apply viterbi to binned long capture
# -> look at mock_continuous_signal.ipynb snippets below

# capture long video, slowly changing the frequency throughout (up then back down)
# bin into 30 second slices, or 40 bins?
# fourier transform
# viterbi to recover meandering frequency

# grid is down, now apply viterbi!

In [ ]:
# given a saved video called expt_5.mp4
# instead of splicing into 40 smaller videos, e.g. expt_5_xx.mp4
# just create one time series and split it later
filename = 'expt_0.mp4'
long_signal, fps = series_at_point(filename,return_series=True)
total_frames = len(long_signal)
# i.e. number of bins/chunks
long_timesteps = 5
bin_total_frames = total_frames//long_timesteps

# tosses away remainder
bin_signals = [long_signal[i: i+bin_total_frames]
               for i in range(0, total_frames-bin_total_frames, bin_total_frames)]

# form up the grid
grid = np.zeros((bin_total_frames//2,long_timesteps))

for i,signal in enumerate(bin_signals):
    col = fourier_spectrum(signal, fps)[0]
    grid[:,i] = col

In [ ]:
"""stitches frequency distribution of sinusoidal signals into a
grid with the frequency slowly changing (meandering) through time,
then applies viterbi's algorithm to try to recover the frequency path"""
# long time as opposed to the short 0 to 2*pi interval of each signal
long_timesteps = 100
bin_time = np.linspace(0,1,long_timesteps)
# meander is the long scale change in the sine frequency
meander_amp = 20
meander_decay = 2
meander_freq = 2
meander = lambda x: meander_amp*(
    np.exp(-x*meander_decay)*
    np.sin(meander_freq*2*np.pi*x))
# connections back a timestep made for all indicies with plus/minus scanning_range
# this limits how much the viterbi path will change frequency at each timestep made
scanning_range = 10
initial_frequency = 20

# halve total number of points due to fft on real function
grid = np.zeros((SineSignal.n_t//2,long_timesteps))
# print('grid is:',grid.shape)
# signal meanders from the initial frequency
wandering_freqs = (initial_frequency+meander(bin_time))
# post_freq is the maximum frequency in the column
# if the resultant plot is exact, then the recovery is perfect
post_freq = []

for i,f in tqdm(enumerate(wandering_freqs)):
    thing_f = SineSignal(f)
    col = thing_f.freq_prob
    # [i][j] same as [:,j][i] same as [i,j]
    grid[:,i] = col
    post_freq.append(thing_f.inj_freq[col.argmax()])

# normalised grid, trying to maximise produce of values
ngrid  = grid/np.max(grid)
# logarithm avoids underflow, maximise sum of log(value)'s
lngrid = np.log(ngrid)

score_grid  = np.copy(lngrid)
pathfinder_flag = len(lngrid[:,0]) #=500
# pathfinder stores the survivor paths, to allow back-tracking through
pathfinder = np.full(np.shape(lngrid), pathfinder_flag)
# pathfinder flag+1 for reaching the first, 0-index column        
pathfinder[:,0] = pathfinder_flag+1       

# the viterbi algorithm, through time finding the best path to each node
# see: https://www.youtube.com/watch?v=6JVqutwtzmo
for j in tqdm(range(1,long_timesteps)): #range(100)
    for i in range(len(score_grid[:,j])): #range(500)
        # index values for where to look relative to i in previous column
        k_a = max(0, i-scanning_range) 
        k_b = min(len(score_grid[:,j-1])-1,
                  i+scanning_range)
        #print(k_a,k_b)
        window = score_grid[:,j-1][k_a:k_b+1]
        # find the best thing nearby in the previous column ...
        window_score = np.max(window)
        window_ref   = k_a+np.argmax(window)
        # ... and take note of it, summing the log(value)'s
        score_grid[i][j] += window_score
        pathfinder[i][j] = window_ref 

# look at the very last column, and find the best ending for the path
best_score  = np.max(score_grid[:,-1])
best_end = np.argmax(score_grid[:,-1])
# now need to retrace the steps through the grid
best_path_back = np.full(long_timesteps,pathfinder_flag+2)
best_path_back[-1] = best_end

# path_grid is the binary image of the viterbi path taken
path_grid = np.zeros(np.shape(ngrid))
tmp_path = pathfinder[best_end][-1]

for j in tqdm(reversed(range(0,long_timesteps-1))):
    path_grid[tmp_path][j] = 1
    # take pathfinder value in current step and follow it backwards
    best_path_back[j] = tmp_path    
    tmp_path = pathfinder[tmp_path][j]

# make sure we got all the way home
assert tmp_path == pathfinder_flag+1

In [ ]:
"""saves plots of meandering frequency, the signal grid, and the recovered viterbi path """
# need ngrid, lngrid, path_grid
plt.figure(figsize=(7,14))
plt.imshow(ngrid, cmap='viridis')
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')
cbar = plt.colorbar() 
cbar.set_label('frequency probability distribution')
plt.title('grid of signals in frequency domain as frequency changes')
plt.ylabel('signal frequency bins')
plt.xlabel('long time bins')
plt.savefig('expt_signal_grid_raw.pdf',bbox_inches='tight')
plt.clf()

plt.figure(figsize=(7,14))
plt.imshow(lngrid, cmap='viridis')
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')        
cbar = plt.colorbar() 
cbar.set_label('log(frequency) probability distribution')
plt.title('grid of signals in log(frequency) domain as frequency changes')
plt.ylabel('signal frequency bins')
plt.xlabel('long time bins')
plt.savefig('expt_lnwandering.pdf',bbox_inches='tight')
plt.clf()

plt.figure(figsize=(7,14))
plt.imshow(path_grid, cmap='viridis')
plt.gca().xaxis.tick_top()
plt.gca().xaxis.set_label_position('top')        
plt.title('viterbi path through signal frequency grid')
plt.ylabel('signal frequency bins')
plt.xlabel('long time bins')
plt.savefig('expt_viterbi_path.pdf',bbox_inches='tight')
plt.clf()

In [ ]:
# track manually found first ring points
expt_nums = '4_0209','4_0580','4_0581','4_1099','4_2014'
points = (200,250),(230,270),(240,250),(220,310),(230,320)
# zexpt_0: y, x = 280, 355

for i in range(len(expt_nums)):
    series_at_point('expt_{}.mp4'.format(expt_nums[i]), points[i],
                    produce_plot=True, out_plot_name='expt_{}.pdf'.format(expt_nums[i]))

In [ ]:
# """tracks points in video"""
# openCV image tracking tutorial
# https://opencv-python-tutroals.readthedocs.io/en/latest/
# py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html
# https://stackoverqflow.com/questions/43063320/cv2-calcopticalflowpyrlk-adding-new-points

# cap = cv2.VideoCapture('noisy_tracking_test.mp4')
cap = cv2.VideoCapture('expt_1_shake.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
pbar = tqdm(total=total_frames-1)

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# # params for ShiTomasi corner detection,
# feature_params = dict( maxCorners = 100,
#                        qualityLevel = 0.3,
#                        minDistance = 7,
#                        blockSize = 7 )
# p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# use pinta to get values for selected points
# tracking_test
# p0 = np.array([[1093,548],[1226,544],[1344,547],[1439,553]],np.float32)
# expt_0: 280,355
# p0 = np.array([[215,275]]).astype('float32')
# expt_1: 453,943 <-centres
p0 = np.array([[735,455]]).astype('float32')
# expt_2: 453,946
# p0 = np.array([[720,450]]).astype('float32')
# expt_3: 458,938
# p0 = np.array([[734,450]]).astype('float32')

# y, x
# ring_centre = 280, 355 #expt_0
ring_centre = 455, 940 #expt_1,2,3

tracked_points = []
# print(p0)
# 427 175 , 401 187
# p0 = np.array([[[427,175]],[[401,187]]])

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
ret, frame = cap.read()

after_first = False

while ret:
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    #print(p1,p0,st,err)
    good_new = p1#[st==1]
    good_old = p0#[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    #cv2.imshow('title',zoom(img))
    #if cv2.waitKey(15) & 0xff == ord('q'):
    #    break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    pbar.update(1)
    
    if after_first:
        tracked_points.append([p[0] for p in p1])    
    else:
        after_first = True
    
    ret,frame = cap.read()

tracked_points = np.array(tracked_points)
pbar.close()
cv2.destroyAllWindows()
cap.release()

# ring_centre = y, x
tracker_0 = tracked_points#[:,:,0]
dist_to_ring_centre = lambda y, x: ((y-ring_centre[0])**2+(x-ring_centre[1])**2)**0.5
tracker_0_radius = np.array([dist_to_ring_centre(*p[0]) for p in tracker_0])

fourier_plot(tracker_0_radius, fps, 'expt_1.pdf')

In [ ]:
# """save a zoomed version of the video"""
cap = cv2.VideoCapture('expt_0.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
pbar = tqdm(total=total_frames)

# width: 600-1400, height: 0-550
ret, frame = cap.read()
zframe = frame[0:550,600:1400]

# width, height
frame_size = zframe.shape[0:2][::-1]

out = cv2.VideoWriter('zexpt_0.mp4',cv2.VideoWriter_fourcc(*'mp4v'),fps,frame_size)

while ret:
    zframe = frame[0:550,600:1400]    
    out.write(np.uint8(zframe))    
   
    ret, frame = cap.read()
    
    frame_count += 1
    pbar.update(1)

pbar.close()
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# """creates mock rings"""
# https://www.niser.ac.in/sps/sites/default/files/basic_page/Michelson%20Interferometer_P744%20-%20Optics.pdf
# maximum when 2*d*cos(theta) = n*lambda
# cos[theta] = n*(lambda/(2*d[t]))
# phys_arg is ultimately time varying, lambda/(2*d[t])
phys_arg = 0.019
total_rings = int(1/phys_arg)
angles = np.arccos([n*phys_arg for n in range(1,total_rings+1)])
# print(angles*180/np.pi)

# pretend that source is 1 away from centre of screen of height 2
# tan(theta) gives then gives the fractional radius
frac_radii = np.tan(angles)
frac_radii = frac_radii[frac_radii < 1]
# print(frac_radii)

# write video of rings
fps = 30
duration = 20
total_frames = 30*5
# caution: array[1], array[0]
frame_size = 512, 512
frame_array_size = 512, 512, 3

out = cv2.VideoWriter('green_rings.mp4',cv2.VideoWriter_fourcc(*'mp4v'),fps,frame_size)

for i in tqdm(range(total_frames)):
    out_frame = np.full(frame_array_size, 255).astype('uint8')
    #out_frame[:,i] = [255,255,255]
    #out_frame = np.uint8(cv2.add(frameA,frameB))

    # cv.Circle(img, center, radius, color, thickness=1) 
    circle_pos_height = int(frame_size[1]/2)
    circle_pos_width  = int(frame_size[0]/2)
    min_frame_dim = int(np.min(frame_size)/2)
    oscillation_amp = 5
    oscillation_speed = i/(2*np.pi)
    
    for r in frac_radii:
        circle_radius = int(r*min_frame_dim + oscillation_amp*np.sin(oscillation_speed))
        cv2.circle(out_frame,(circle_pos_width,circle_pos_height),circle_radius,[0,255,0]) 
    
    #cv2.imshow('title',out_frame)
    #if cv2.waitKey(30) & 0xFF == ord('q'):
    #    break
    
    out.write(out_frame)

out.release()
cv2.destroyAllWindows()

In [ ]:
# """adds noise to a video"""
cap = cv2.VideoCapture('green_rings.mp4')
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
pbar = tqdm(total=total_frames-1)

ret, frame = cap.read()

fps = cap.get(cv2.CAP_PROP_FPS)
# width, height
frame_size = frame.shape[0:2][::-1]

out = cv2.VideoWriter('noisy_green_rings.mp4',cv2.VideoWriter_fourcc(*'mp4v'),fps,frame_size)

while ret:
    
    #out_frame = cv2.flip(frame,1)
    noise_level = 10
    noisy_frame = np.random.normal(0,noise_level,frame.shape).astype('uint8')
    out_frame = cv2.add(frame,noisy_frame)
    #out_frame = cv2.add(frame,frame*noisy_frame)
    #cv2.normalize(noisy_frame, noisy_frame, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    #noisy_image = noisy_image.astype(np.uint8)
    #out_frame[out_frame < 0] = 0
    #out_frame[out_frame > 255] = 255
    out.write(np.uint8(out_frame))
    
    #cv2.imshow('noisy vid',zoom(out_frame))
    #if cv2.waitKey(30) & 0xFF == ord('q'):
    #    break    
    
    ret, frame = cap.read()
    pbar.update(1)

pbar.close()
cap.release()
out.release()
cv2.destroyAllWindows()